### Задача прогнозирования
Разберём задачу прогнозирования временных рядов.

Задача прогнозирования временного ряда состоит в построении модели, которая по историческим данным предскажет будущие значения временного ряда.
Промежуток времени в будущем, на который строится прогноз, называется горизонтом прогнозирования (англ.  forecast horizon). В задачах этой темы он будет равен одному шагу.

Если значения временного ряда, или функция x(t), где t — время, — это числа, то перед вами задача регрессии для временных рядов; если категории — задача классификации.

Например, к задачам прогнозирования рядов относятся предсказания:
- количества заказов такси на следующий час,
- объёма доставок курьерской службы на следующий день,
- объёма спроса на товары в интернет-магазине на следующей неделе.

По исходным данным создадим обучающую (train) и тестовую (test) выборки.

Перемешивать выборки в задаче прогнозирования временного ряда нельзя.

Данные обучающей выборки должны предшествовать данным тестовой. Иначе тестирование модели будет некорректным: модель не должна обучаться на данных из будущего.
<img src ='https://github.com/trisha00001/file/blob/main/traintest_1581893884.jpg?raw=true'>

Функция train_test_split() из модуля sklearn.model_selection по умолчанию перемешивает данные. Поэтому укажем аргумент shuffle (с англ. «перетасовывать») равным False, чтобы разделить данные корректно:

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.Series([0.1, 0.5, 2.3, 1.2, 1.5])
train, test = train_test_split(data, shuffle=False, test_size=0.2)
print('Обучающая выборка:')
print(train)
print('Тестовая выборка:')
print(test)

Обучающая выборка:
0    0.1
1    0.5
2    2.3
3    1.2
dtype: float64
Тестовая выборка:
4    1.5
dtype: float64


Таким же образом данные делятся на три выборки: обучающую, валидационную и тестовую.

## Задача

Разбейте датасет о потреблении электроэнергии на обучающую и тестовую выборки в соотношении 4:1. Возьмите данные за доступное время.

Напечатайте на экране минимальные и максимальные значения индексов выборок (уже в прекоде). Они нужны, чтобы убедиться в корректности деления.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('https://raw.githubusercontent.com/trisha00001/file/main/energy_consumption.csv', index_col=[0], parse_dates=[0])
data.sort_index(inplace=True)
data = data.resample('1D').sum()

train, test = train_test_split(data, shuffle=False, test_size=0.2)
print(train.index.min(), train.index.max())
print(test.index.min(), test.index.max())

2002-01-01 00:00:00 2015-04-09 00:00:00
2015-04-10 00:00:00 2018-08-03 00:00:00


###Качество прогноза
Научимся измерять качество предсказания временных рядов и проверять модели на адекватность.

Обучим модель с горизонтом прогнозирования в один день. Такие модели пригодятся в автоматизации принятия технических решений. Например, в задаче энергопотребления модель поможет изменять режим работы генераторов автоматически.

Чтобы проверять качество моделей в наших задачах, возьмём метрику MAE. Её можно легко интерпретировать.

Спрогнозировать временные ряды без обучения можно двумя способами:
1. Все значения тестовой выборки предсказываются одним и тем же числом (константой). Для метрики MAE — это медиана.
2. Новое значение x(t) прогнозируется предыдущим значением ряда, то есть x(t-1). Этот способ не зависит от метрики.

1. Оцените модель первым способом — прогнозом константой. Дневной объём электропотребления предскажите медианой, сохраните значения в переменной pred_median и найдите для этого прогноза значение MAE.

В прекоде указан средний объём электропотребления, чтобы вы смогли соотнести его со значением метрики MAE.

Напечатайте на экране значения среднего объёма электропотребления и метрики MAE (уже в прекоде).

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

data = pd.read_csv('https://raw.githubusercontent.com/trisha00001/file/main/energy_consumption.csv', index_col=[0], parse_dates=[0])
data.sort_index(inplace=True)
data = data.resample('1D').sum()

train, test = train_test_split(data, shuffle=False, test_size=0.2)


print("Средний объём электропотребления в день:", test['PJME_MW'].mean())

pred_median = np.ones(test.shape) * train['PJME_MW'].median()
print("MAE:", mean_absolute_error(test ,pred_median))

Средний объём электропотребления в день: 745523.4529702971
MAE: 96625.08333333333


2. Оцените модель вторым способом — предыдущим значением ряда. Предскажите дневной объём электропотребления и найдите для этого прогноза значение MAE.

В прекоде указан средний объём электропотребления, чтобы вы смогли соотнести его со значением метрики MAE.

Напечатайте на экране значения среднего объёма электропотребления и метрики MAE (уже в прекоде).

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

data = pd.read_csv('https://raw.githubusercontent.com/trisha00001/file/main/energy_consumption.csv', index_col=[0], parse_dates=[0])
data.sort_index(inplace=True)
data = data.resample('1D').sum()

train, test = train_test_split(data, shuffle=False, test_size=0.2)

print("Средний объём электропотребления в день:", test['PJME_MW'].mean())

pred_previous = test.shift()
# заполняем первое значение
pred_previous.iloc[0] = train.iloc[-1]
print("MAE:", mean_absolute_error(test, pred_previous))

Средний объём электропотребления в день: 745523.4529702971
MAE: 44941.65924092409


### Создание признаков
Создадим признаки для горизонта прогнозирования в один шаг.
Напишем функцию для создания признаков. Разберём каждый тип признаков:
1. Календарные признаки (англ. calendar features)
Во многих данных тренды и сезонность привязаны к конкретной дате. Тип datetime64 в Pandas уже содержит нужную информацию, осталось лишь представить её как отдельные столбцы. Рассмотрим пример:

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/trisha00001/file/main/energy_consumption.csv', index_col=[0], parse_dates=[0])
data.sort_index(inplace=True)
data = data.resample('1D').sum()

# признак, в котором хранится год как число
data['year'] = data.index.year

# признак, в котором хранится день недели как число
data['dayofweek'] = data.index.dayofweek

print(data.head(10))

             PJME_MW  year  dayofweek
Datetime                             
2002-01-01  714857.0  2002          1
2002-01-02  822277.0  2002          2
2002-01-03  828285.0  2002          3
2002-01-04  809171.0  2002          4
2002-01-05  729723.0  2002          5
2002-01-06  727766.0  2002          6
2002-01-07  800012.0  2002          0
2002-01-08  824710.0  2002          1
2002-01-09  810628.0  2002          2
2002-01-10  755317.0  2002          3


2. «Отстающие значения» (англ. lag features)
Предыдущие значения временного ряда подскажут, будет ли функция x(t) расти или уменьшаться. Получим отстающие значения знакомой функцией shift():


In [ ]:
data['lag_1'] = data['PJME_MW'].shift(1)
data['lag_2'] = data['PJME_MW'].shift(2)
data['lag_3'] = data['PJME_MW'].shift(3)

print(data.head(10))

             PJME_MW  year  dayofweek     lag_1     lag_2     lag_3
Datetime                                                           
2002-01-01  714857.0  2002          1       NaN       NaN       NaN
2002-01-02  822277.0  2002          2  714857.0       NaN       NaN
2002-01-03  828285.0  2002          3  822277.0  714857.0       NaN
2002-01-04  809171.0  2002          4  828285.0  822277.0  714857.0
2002-01-05  729723.0  2002          5  809171.0  828285.0  822277.0
2002-01-06  727766.0  2002          6  729723.0  809171.0  828285.0
2002-01-07  800012.0  2002          0  727766.0  729723.0  809171.0
2002-01-08  824710.0  2002          1  800012.0  727766.0  729723.0
2002-01-09  810628.0  2002          2  824710.0  800012.0  727766.0
2002-01-10  755317.0  2002          3  810628.0  824710.0  800012.0


Для первых дат есть не все отстающие значения, поэтому в этих строках стоят NaN.
3. Скользящее среднее
Скользящее среднее как признак задаёт общий тренд временного ряда. Повторим, как его вычислять:

In [ ]:
data['rolling_mean'] = data['PJME_MW'].rolling(5).mean()

print(data.head(10))

             PJME_MW  year  dayofweek     lag_1     lag_2     lag_3  \
Datetime                                                              
2002-01-01  714857.0  2002          1       NaN       NaN       NaN   
2002-01-02  822277.0  2002          2  714857.0       NaN       NaN   
2002-01-03  828285.0  2002          3  822277.0  714857.0       NaN   
2002-01-04  809171.0  2002          4  828285.0  822277.0  714857.0   
2002-01-05  729723.0  2002          5  809171.0  828285.0  822277.0   
2002-01-06  727766.0  2002          6  729723.0  809171.0  828285.0   
2002-01-07  800012.0  2002          0  727766.0  729723.0  809171.0   
2002-01-08  824710.0  2002          1  800012.0  727766.0  729723.0   
2002-01-09  810628.0  2002          2  824710.0  800012.0  727766.0   
2002-01-10  755317.0  2002          3  810628.0  824710.0  800012.0   

            rolling_mean  
Datetime                  
2002-01-01           NaN  
2002-01-02           NaN  
2002-01-03           NaN  
2002-01-04  

Скользящее среднее в моменте t учитывает текущее значение ряда x(t). Это некорректно: целевой признак «убежал» в признаки. Вычисление скользящего среднего не должно включать в себя текущее значение ряда.

Задание

1. Напишите функцию make_features() (англ. «создать признаки»), чтобы прибавить к таблице четыре новых календарных признака: год, месяц, день и день недели. Имена столбцов должны быть такие: 'year', 'month', 'day', 'dayofweek'.

Примените функцию к таблице и напечатайте на экране её первые пять строк (уже в прекоде).

In [ ]:
import pandas as pd
import numpy as np


data = pd.read_csv('https://raw.githubusercontent.com/trisha00001/file/main/energy_consumption.csv', index_col=[0], parse_dates=[0])
data.sort_index(inplace=True)
data = data.resample('1D').sum()

def make_features(data):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek


make_features(data)
print(data.head())

             PJME_MW  year  month  day  dayofweek
Datetime                                         
2002-01-01  714857.0  2002      1    1          1
2002-01-02  822277.0  2002      1    2          2
2002-01-03  828285.0  2002      1    3          3
2002-01-04  809171.0  2002      1    4          4
2002-01-05  729723.0  2002      1    5          5


2. Вычислите отстающие значения. В функцию make_features() добавьте новый аргумент max_lag, который задаст максимальный размер отставания. Новые признаки назовите: 'lag_1', 'lag_2' — и до величины max_lag.

Примените функцию к таблице и напечатайте на экране её первые пять строк (уже в прекоде).

In [ ]:
import pandas as pd
import numpy as np


data = pd.read_csv('https://raw.githubusercontent.com/trisha00001/file/main/energy_consumption.csv', index_col=[0], parse_dates=[0])
data.sort_index(inplace=True)
data = data.resample('1D').sum()
def make_features(data, max_lag):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek

    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['PJME_MW'].shift(lag)



make_features(data, 4)
print(data.head())

             PJME_MW  year  month  day  dayofweek     lag_1     lag_2  \
Datetime                                                                
2002-01-01  714857.0  2002      1    1          1       NaN       NaN   
2002-01-02  822277.0  2002      1    2          2  714857.0       NaN   
2002-01-03  828285.0  2002      1    3          3  822277.0  714857.0   
2002-01-04  809171.0  2002      1    4          4  828285.0  822277.0   
2002-01-05  729723.0  2002      1    5          5  809171.0  828285.0   

               lag_3     lag_4  
Datetime                        
2002-01-01       NaN       NaN  
2002-01-02       NaN       NaN  
2002-01-03       NaN       NaN  
2002-01-04  714857.0       NaN  
2002-01-05  822277.0  714857.0  


3. Вычислите скользящее среднее и добавьте его как признак 'rolling_mean'. В функцию make_features() добавьте новый аргумент rolling_mean_size, который задаст ширину окна. Текущее значение ряда для расчёта скользящего среднего применять нельзя.

Примените функцию к таблице и напечатайте на экране её первые пять строк (уже в прекоде).

In [ ]:
import pandas as pd
import numpy as np


data = pd.read_csv('https://raw.githubusercontent.com/trisha00001/file/main/energy_consumption.csv', index_col=[0], parse_dates=[0])
data.sort_index(inplace=True)
data = data.resample('1D').sum()

def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek

    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['PJME_MW'].shift(lag)

    data['rolling_mean'] = data['PJME_MW'].shift().rolling(4).mean()


make_features(data, 4, 4)
print(data.head())

             PJME_MW  year  month  day  dayofweek     lag_1     lag_2  \
Datetime                                                                
2002-01-01  714857.0  2002      1    1          1       NaN       NaN   
2002-01-02  822277.0  2002      1    2          2  714857.0       NaN   
2002-01-03  828285.0  2002      1    3          3  822277.0  714857.0   
2002-01-04  809171.0  2002      1    4          4  828285.0  822277.0   
2002-01-05  729723.0  2002      1    5          5  809171.0  828285.0   

               lag_3     lag_4  rolling_mean  
Datetime                                      
2002-01-01       NaN       NaN           NaN  
2002-01-02       NaN       NaN           NaN  
2002-01-03       NaN       NaN           NaN  
2002-01-04  714857.0       NaN           NaN  
2002-01-05  822277.0  714857.0      793647.5  


### Обучение модели

Обучим линейную регрессию с учётом новых признаков.

Разделим данные на обучающую и тестовую выборки. Отстающие значения и скользящее среднее вычисляются по прошлым данным. Признаки для первых значений тестовой выборки находятся в конце обучающей выборки.

Получить признаки для первых значений обучающей выборки нельзя: исторических данных по ним нет. В предыдущих задачах значения этих признаков были NaN. Их нужно удалить.

Задача

1. Разбейте датасет о потреблении электроэнергии на обучающую и тестовую выборки в соотношении 4:1. Вам нужны данные за всё время. Из обучающей выборки удалите строки с пропусками.

Напечатайте на экране размеры обучающей и тестовой выборки (уже в прекоде).

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


data = pd.read_csv('https://raw.githubusercontent.com/trisha00001/file/main/energy_consumption.csv', index_col=[0], parse_dates=[0])
data.sort_index(inplace=True)
data = data.resample('1D').sum()

def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek

    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['PJME_MW'].shift(lag)

    data['rolling_mean'] = data['PJME_MW'].shift().rolling(rolling_mean_size).mean()

# мы выбрали произвольные значения аргументов
make_features(data, 1, 1)

train, test = train_test_split(data, shuffle=False, test_size=0.2)
train = train.dropna()

print(train.shape)
print(test.shape)

(4846, 7)
(1212, 7)


2. В выборке выделите признаки и целевой признак. На них обучите линейную регрессию и сохраните её в переменной model. Затем напечатайте на экране значения MAE для обучающей и тестовой выборок (уже в прекоде). Подберите аргументы функции make_features() так, чтобы значение MAE на тестовой выборке было не больше 37 000.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


data = pd.read_csv('https://raw.githubusercontent.com/trisha00001/file/main/energy_consumption.csv', index_col=[0], parse_dates=[0])
data.sort_index(inplace=True)
data = data.resample('1D').sum()

def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek

    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['PJME_MW'].shift(lag)

    data['rolling_mean'] = data['PJME_MW'].shift().rolling(rolling_mean_size).mean()


make_features(data, 1, 1)

train, test = train_test_split(data, shuffle=False, test_size=0.2)
train = train.dropna()

features_train  = train.drop('PJME_MW', axis=1)
target_train = train[['PJME_MW']]
features_test = test.drop('PJME_MW', axis=1)
target_test = test[['PJME_MW']]

model = LinearRegression()

model1 = model.fit(features_train, target_train)
predictions1 = model1.predict(features_test) # получим предсказания модели
result1 = mean_absolute_error(target_test,predictions1)

model2 = model.fit(features_test, target_test)
predictions2 = model2.predict(features_train) # получим предсказания модели
result2 = mean_absolute_error(target_train,predictions2)


print("MAE обучающей выборки:", result1)
print("MAE тестовой выборки: ", result2)

MAE обучающей выборки: 39180.717976797976
MAE тестовой выборки:  36246.52420637463
